In [25]:
# Logistic regression

import pandas as pd
import numpy as np
from collections import defaultdict
import math
import matplotlib.pyplot as plt
from sklearn . model_selection import train_test_split

In [27]:
# Importing the data from the csv file
training_data = pd.read_csv("../siren_data_train.csv", sep=",")
training_data['heard'] = training_data['heard'].replace({'hearing': 1, 'not_hearing': 0})
print('Data imported!')

# Creating training set and test set
# Creating X - everything about the column 'heard'
X = training_data.drop(['heard'], axis = 1)

# Creating Y - the column 'heard'
Y = training_data['heard']

# Splitting the data into 75% for training and 25% for testing
# Stratify - a sampling method that reduces the sampling error in cases where the pop. can be partitioned into subgroups
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 1477, stratify = Y)
print("Data splitted!")

Data imported!
      near_fid        near_x        near_y  near_angle  building      xcoor  \
183       4022  1.994137e+06  8.298690e+06  -40.363174         0  1993750.0   
5244      3460  1.418313e+06  7.579746e+06  112.548413         1  1419318.0   
979        538  1.964992e+06  8.371630e+06   59.088381         0  1964697.0   
4531      3450  1.416968e+06  7.566548e+06 -131.032953         1  1417219.0   
3855      1826  1.326432e+06  7.904828e+06   31.497378         1  1325560.0   
...        ...           ...           ...         ...       ...        ...   
3848      1510  1.445371e+06  7.477707e+06   28.045037         0  1445121.0   
1285       141  2.014293e+06  8.244952e+06   40.903749         0  2013525.0   
3881      2430  1.833571e+06  8.315400e+06  148.726758         0  1833881.0   
4371      2661  1.796142e+06  8.817041e+06   98.540604         0  1799944.0   
1830      2344  1.674021e+06  8.369899e+06    8.298469         0  1673465.0   

        ycoor  noise  in_vehicle  as

In [ ]:
# Training the model Logistic Regression

# Application to data:
# Logistic regression makes the assumption that there is a linear relationship between input variables
# and the probability of it belonging to the positive class. An input variable x1 increase p(x1) will increase or decrease